# (Optional) Colab Setup
If you aren't using Colab, you can delete the following code cell. This is just to help students with mounting to Google Drive to access the other .py files and downloading the data, which is a little trickier on Colab than on your local machine using Jupyter. 

In [ ]:
# you will be prompted with a window asking to grant permissions
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
# fill in the path in your Google Drive in the string below. Note: do not escape slashes or spaces
import os
datadir = "/content/assignment1"
if not os.path.exists(datadir):
  !ln -s "/content/drive/My Drive/YOUR PATH HERE/assignment1/" $datadir
os.chdir(datadir)
!pwd

In [4]:
# downloading Fashion-MNIST
import os
os.chdir(os.path.join(datadir,"fashion-mnist/"))
!chmod +x ./get_data.sh
!./get_data.sh
os.chdir(datadir)

# Imports

In [2]:
import random
import numpy as np
from data_process import get_FASHION_data, get_RICE_data
from scipy.spatial import distance
from models import Perceptron, SVM, Softmax, Logistic
from kaggle_submission import output_submission_csv
%matplotlib inline

# For auto-reloading external modules
# See http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

# Loading Fashion-MNIST

In the following cells we determine the number of images for each split and load the images.
<br /> 
TRAIN_IMAGES + VAL_IMAGES = (0, 60000]
, TEST_IMAGES = 10000

In [3]:
# You can change these numbers for experimentation
# For submission we will use the default values 
TRAIN_IMAGES = 50000
VAL_IMAGES = 10000
normalize = True

In [4]:
data = get_FASHION_data(TRAIN_IMAGES, VAL_IMAGES, normalize=normalize)
X_train_fashion, y_train_fashion = data['X_train'], data['y_train']
X_val_fashion, y_val_fashion = data['X_val'], data['y_val']
X_test_fashion, y_test_fashion = data['X_test'], data['y_test']
n_class_fashion = len(np.unique(y_test_fashion))

# Loading Rice

In [5]:
# loads train / test / val splits of 80%, 20%, 20% 
data = get_RICE_data()
X_train_RICE, y_train_RICE = data['X_train'], data['y_train']
X_val_RICE, y_val_RICE = data['X_val'], data['y_val']
X_test_RICE, y_test_RICE = data['X_test'], data['y_test']
n_class_RICE = len(np.unique(y_test_RICE))

print("Number of train samples: ", X_train_RICE.shape[0])
print("Number of val samples: ", X_val_RICE.shape[0])
print("Number of test samples: ", X_test_RICE.shape[0])

Number of train samples:  10911
Number of val samples:  3637
Number of test samples:  3637


### Get Accuracy

This function computes how well your model performs using accuracy as a metric.

In [6]:
def get_acc(pred, y_test):
    return np.sum(y_test == pred) / len(y_test) * 100

# Perceptron

Perceptron has 2 hyperparameters that you can experiment with:
- **Learning rate** - controls how much we change the current weights of the classifier during each update. We set it at a default value of 0.5, but you should experiment with different values. We recommend changing the learning rate by factors of 10 and observing how the performance of the classifier changes. You should also try adding a **decay** which slowly reduces the learning rate over each epoch.
- **Number of Epochs** - An epoch is a complete iterative pass over all of the data in the dataset. During an epoch we predict a label using the classifier and then update the weights of the classifier according to the perceptron update rule for each sample in the training set. You should try different values for the number of training epochs and report your results.

You will implement the Perceptron classifier in the **models/perceptron.py**

The following code: 
- Creates an instance of the Perceptron classifier class 
- The train function of the Perceptron class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy


## Train Perceptron on Fashion-MNIST

In [636]:
for lr in (0.5, 0.05):
    for n_epochs in range(10, 15, 2):
        percept_fashion = Perceptron(n_class_fashion, lr, n_epochs)
        percept_fashion.train(X_train_fashion, y_train_fashion)
        print(lr, n_epochs)
        pred_percept = percept_fashion.predict(X_train_fashion)
        print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_fashion)))
        pred_percept = percept_fashion.predict(X_val_fashion)
        print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_fashion)))
        pred_percept = percept_fashion.predict(X_test_fashion)
        print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_fashion)))
        print()


0.5 10
The training accuracy is given by: 84.460000
The validation accuracy is given by: 82.370000
The testing accuracy is given by: 81.490000

0.5 12
The training accuracy is given by: 84.510000
The validation accuracy is given by: 82.640000
The testing accuracy is given by: 81.680000

0.5 14
The training accuracy is given by: 84.452000
The validation accuracy is given by: 82.540000
The testing accuracy is given by: 81.410000

0.05 10
The training accuracy is given by: 84.278000
The validation accuracy is given by: 82.380000
The testing accuracy is given by: 81.230000

0.05 12
The training accuracy is given by: 84.370000
The validation accuracy is given by: 82.560000
The testing accuracy is given by: 81.380000

0.05 14
The training accuracy is given by: 84.542000
The validation accuracy is given by: 82.490000
The testing accuracy is given by: 81.390000



In [638]:
lr = 0.05
n_epochs = 10
percept_fashion = Perceptron(n_class_fashion, lr, n_epochs)
percept_fashion.train(X_train_fashion, y_train_fashion)

In [639]:
pred_percept = percept_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_fashion)))

The training accuracy is given by: 84.384000


### Validate Perceptron on Fashion-MNIST

In [640]:
pred_percept = percept_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_fashion)))

The validation accuracy is given by: 82.500000


### Test Perceptron on Fashion-MNIST

In [641]:
pred_percept = percept_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_fashion)))

The testing accuracy is given by: 81.470000


### Perceptron_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy, output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [331]:
output_submission_csv('kaggle/perceptron_submission_fashion.csv', percept_fashion.predict(X_test_fashion))

## Train Perceptron on Rice

In [637]:
for lr in (0.5, 0.05):
    for n_epochs in range(10, 15, 2):
        percept_RICE = Perceptron(n_class_RICE, lr, n_epochs)
        percept_RICE.train(X_train_RICE, y_train_RICE)
        print(lr, n_epochs)
        pred_percept = percept_RICE.predict(X_train_RICE)
        print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_RICE)))
        pred_percept = percept_RICE.predict(X_val_RICE)
        print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_RICE)))
        pred_percept = percept_RICE.predict(X_test_RICE)
        print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_RICE)))
        print()

0.5 10
The training accuracy is given by: 99.743378
The validation accuracy is given by: 99.725048
The testing accuracy is given by: 99.560077

0.5 12
The training accuracy is given by: 99.770873
The validation accuracy is given by: 99.752543
The testing accuracy is given by: 99.587572

0.5 14
The training accuracy is given by: 99.770873
The validation accuracy is given by: 99.752543
The testing accuracy is given by: 99.587572

0.05 10
The training accuracy is given by: 99.807534
The validation accuracy is given by: 99.807534
The testing accuracy is given by: 99.670058

0.05 12
The training accuracy is given by: 99.789204
The validation accuracy is given by: 99.835029
The testing accuracy is given by: 99.587572

0.05 14
The training accuracy is given by: 99.807534
The validation accuracy is given by: 99.835029
The testing accuracy is given by: 99.752543



In [642]:
lr = 0.05
n_epochs = 14
percept_RICE = Perceptron(n_class_RICE, lr, n_epochs)
percept_RICE.train(X_train_RICE, y_train_RICE)

In [643]:
pred_percept = percept_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_percept, y_train_RICE)))

The training accuracy is given by: 99.816699


### Validate Perceptron on Rice

In [644]:
pred_percept = percept_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_percept, y_val_RICE)))

The validation accuracy is given by: 99.862524


### Test Perceptron on Rice

In [645]:
pred_percept = percept_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_percept, y_test_RICE)))

The testing accuracy is given by: 99.697553


# Support Vector Machines (with SGD)

Next, you will implement a "soft margin" SVM. In this formulation you will maximize the margin between positive and negative training examples and penalize margin violations using a hinge loss.

We will optimize the SVM loss using SGD. This means you must compute the loss function with respect to model weights. You will use this gradient to update the model weights.

SVM optimized with SGD has 3 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Epochs** - similar to as defined above in Perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case it is a coefficient on the term which maximizes the margin. You could try different values. The default value is set to 0.05.

You will implement the SVM using SGD in the **models/svm.py**

The following code: 
- Creates an instance of the SVM classifier class 
- The train function of the SVM class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train SVM on Fashion-MNIST

In [658]:
for lr in (0.5, 0.05):
    for n_epochs in (10, 12):
        for reg_const in (0.05, 0.1, 0.15):
            svm_fashion = SVM(n_class_fashion, lr, n_epochs, reg_const)
            svm_fashion.train(X_train_fashion, y_train_fashion)
            pred_svm = svm_fashion.predict(X_train_fashion)
            print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_fashion)))  
            pred_svm = svm_fashion.predict(X_val_fashion)
            print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_fashion))) 
            pred_svm = svm_fashion.predict(X_test_fashion)
            print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_fashion)))

The training accuracy is given by: 83.978000
The validation accuracy is given by: 82.800000
The testing accuracy is given by: 81.570000
The training accuracy is given by: 83.658000
The validation accuracy is given by: 82.710000
The testing accuracy is given by: 81.700000
The training accuracy is given by: 83.496000
The validation accuracy is given by: 82.330000
The testing accuracy is given by: 81.330000
The training accuracy is given by: 83.944000
The validation accuracy is given by: 83.050000
The testing accuracy is given by: 81.670000
The training accuracy is given by: 83.738000
The validation accuracy is given by: 82.680000
The testing accuracy is given by: 81.710000
The training accuracy is given by: 83.296000
The validation accuracy is given by: 82.480000
The testing accuracy is given by: 81.370000
The training accuracy is given by: 83.874000
The validation accuracy is given by: 82.290000
The testing accuracy is given by: 81.350000
The training accuracy is given by: 83.436000
The

In [654]:
lr = 0.5
n_epochs = 10
reg_const = 0.05

svm_fashion = SVM(n_class_fashion, lr, n_epochs, reg_const)
svm_fashion.train(X_train_fashion, y_train_fashion)

In [655]:
pred_svm = svm_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_fashion)))

The training accuracy is given by: 83.682000


### Validate SVM on Fashion-MNIST

In [656]:
pred_svm = svm_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_fashion)))

The validation accuracy is given by: 82.510000


### Test SVM on Fashion-MNIST

In [657]:
pred_svm = svm_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_fashion)))

The testing accuracy is given by: 81.440000


### SVM_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [382]:
output_submission_csv('kaggle/svm_submission_fashion.csv', svm_fashion.predict(X_test_fashion))

## Train SVM on Rice

In [660]:
for lr in (0.5, 0.05):
    for n_epochs in (10, 12):
        for reg_const in (0.05, 0.1, 0.15):
            print(lr,n_epochs,reg_const)
            svm_RICE = SVM(n_class_RICE, lr, n_epochs, reg_const)
            svm_RICE.train(X_train_RICE, y_train_RICE)
            pred_svm = svm_RICE.predict(X_train_RICE)
            print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_RICE)))
            pred_svm = svm_RICE.predict(X_val_RICE)
            print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_RICE)))
            pred_svm = svm_RICE.predict(X_test_RICE)
            print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_RICE)))
            print("\n")


0.5 10 0.05
The training accuracy is given by: 99.789204
The validation accuracy is given by: 99.725048
The testing accuracy is given by: 99.670058


0.5 10 0.1
The training accuracy is given by: 99.880854
The validation accuracy is given by: 99.835029
The testing accuracy is given by: 99.862524


0.5 10 0.15
The training accuracy is given by: 99.853359
The validation accuracy is given by: 99.807534
The testing accuracy is given by: 99.780038


0.5 12 0.05
The training accuracy is given by: 99.789204
The validation accuracy is given by: 99.725048
The testing accuracy is given by: 99.670058


0.5 12 0.1
The training accuracy is given by: 99.880854
The validation accuracy is given by: 99.835029
The testing accuracy is given by: 99.862524


0.5 12 0.15
The training accuracy is given by: 99.853359
The validation accuracy is given by: 99.807534
The testing accuracy is given by: 99.780038


0.05 10 0.05
The training accuracy is given by: 99.789204
The validation accuracy is given by: 99.8075

In [661]:
lr = 0.5
n_epochs = 10
reg_const = 0.05

svm_RICE = SVM(n_class_RICE, lr, n_epochs, reg_const)
svm_RICE.train(X_train_RICE, y_train_RICE)

In [662]:
pred_svm = svm_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_svm, y_train_RICE)))

The training accuracy is given by: 99.789204


### Validate SVM on Rice

In [663]:
pred_svm = svm_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_svm, y_val_RICE)))

The validation accuracy is given by: 99.725048


## Test SVM on Rice

In [664]:
pred_svm = svm_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_svm, y_test_RICE)))

The testing accuracy is given by: 99.670058


# Softmax Classifier (with SGD)

Next, you will train a Softmax classifier. This classifier consists of a linear function of the input data followed by a softmax function which outputs a vector of dimension C (number of classes) for each data point. Each entry of the softmax output vector corresponds to a confidence in one of the C classes, and like a probability distribution, the entries of the output vector sum to 1. We use a cross-entropy loss on this sotmax output to train the model. 

Check the following link as an additional resource on softmax classification: http://cs231n.github.io/linear-classify/#softmax

Once again we will train the classifier with SGD. This means you need to compute the gradients of the softmax cross-entropy loss function according to the weights and update the weights using this gradient. Check the following link to help with implementing the gradient updates: https://deepnotes.io/softmax-crossentropy

The softmax classifier has 3 hyperparameters that you can experiment with:
- **Learning rate** - As above, this controls how much the model weights are updated with respect to their gradient.
- **Number of Epochs** - As described for perceptron.
- **Regularization constant** - Hyperparameter to determine the strength of regularization. In this case, we minimize the L2 norm of the model weights as regularization, so the regularization constant is a coefficient on the L2 norm in the combined cross-entropy and regularization objective.

You will implement a softmax classifier using SGD in the **models/softmax.py**

The following code: 
- Creates an instance of the Softmax classifier class 
- The train function of the Softmax class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

## Train Softmax on Fashion-MNIST

In [703]:
for lr in (0.5, 0.05):
    for n_epochs in (10, 12):
        for reg_const in (0.05, 0.1, 0.15):
            print(lr,n_epochs,reg_const)
            softmax_fashion = Softmax(n_class_fashion, lr, n_epochs, reg_const)
            softmax_fashion.train(X_train_fashion, y_train_fashion)
            pred_softmax = softmax_fashion.predict(X_train_fashion)
            print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_fashion)))
            pred_softmax = softmax_fashion.predict(X_val_fashion)
            print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_fashion)))
            pred_softmax = softmax_fashion.predict(X_test_fashion)
            print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_fashion)))
            print("\n")

0.5 10 0.05
The training accuracy is given by: 84.628000
The validation accuracy is given by: 83.160000
The testing accuracy is given by: 82.020000


0.5 10 0.1
The training accuracy is given by: 84.464000
The validation accuracy is given by: 83.140000
The testing accuracy is given by: 81.440000


0.5 10 0.15
The training accuracy is given by: 84.314000
The validation accuracy is given by: 82.870000
The testing accuracy is given by: 81.490000


0.5 12 0.05
The training accuracy is given by: 84.472000
The validation accuracy is given by: 82.710000
The testing accuracy is given by: 82.030000


0.5 12 0.1
The training accuracy is given by: 84.354000
The validation accuracy is given by: 82.750000
The testing accuracy is given by: 81.800000


0.5 12 0.15
The training accuracy is given by: 84.428000
The validation accuracy is given by: 82.920000
The testing accuracy is given by: 81.900000


0.05 10 0.05
The training accuracy is given by: 84.890000
The validation accuracy is given by: 82.7300

In [31]:
lr = 0.5
n_epochs = 10
reg_const = 0。9

softmax_fashion = Softmax(n_class_fashion, lr, n_epochs, reg_const)
softmax_fashion.train(X_train_fashion, y_train_fashion)

In [32]:
pred_softmax = softmax_fashion.predict(X_train_fashion)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_fashion)))

The training accuracy is given by: 84.636000


### Validate Softmax on Fashion-MNIST

In [33]:
pred_softmax = softmax_fashion.predict(X_val_fashion)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_fashion)))

The validation accuracy is given by: 83.340000


### Testing Softmax on Fashion-MNIST

In [34]:
pred_softmax = softmax_fashion.predict(X_test_fashion)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_fashion)))

The testing accuracy is given by: 82.200000


### Softmax_Fashion-MNIST Kaggle Submission

Once you are satisfied with your solution and test accuracy output a file to submit your test set predictions to the Kaggle for Assignment 1 Fashion-MNIST. Use the following code to do so:

In [429]:
output_submission_csv('kaggle/softmax_submission_fashion.csv', softmax_fashion.predict(X_test_fashion))

## Train Softmax on Rice

In [708]:
for lr in (0.5, 0.05):
    for n_epochs in (10, 12):
        for reg_const in (0.05, 0.1, 0.15):
            print(lr,n_epochs,reg_const)
            softmax_RICE = Softmax(n_class_RICE, lr, n_epochs, reg_const)
            softmax_RICE.train(X_train_RICE, y_train_RICE)
            pred_softmax = softmax_RICE.predict(X_train_RICE)
            print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_RICE)))
            pred_softmax = softmax_RICE.predict(X_val_RICE)
            print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_RICE)))
            pred_softmax = softmax_RICE.predict(X_test_RICE)
            print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_RICE)))
            print("\n")

0.5 10 0.05
The training accuracy is given by: 99.844194
The validation accuracy is given by: 99.780038
The testing accuracy is given by: 99.752543


0.5 10 0.1
The training accuracy is given by: 98.808542
The validation accuracy is given by: 98.872697
The testing accuracy is given by: 98.680231


0.5 10 0.15
The training accuracy is given by: 98.377784
The validation accuracy is given by: 98.487765
The testing accuracy is given by: 98.157822


0.5 12 0.05
The training accuracy is given by: 99.844194
The validation accuracy is given by: 99.780038
The testing accuracy is given by: 99.752543


0.5 12 0.1
The training accuracy is given by: 98.762717
The validation accuracy is given by: 98.845202
The testing accuracy is given by: 98.515260


0.5 12 0.15
The training accuracy is given by: 98.643571
The validation accuracy is given by: 98.762717
The testing accuracy is given by: 98.570250


0.05 10 0.05
The training accuracy is given by: 99.230135
The validation accuracy is given by: 99.1751

In [709]:
lr = 0.5
n_epochs = 10
reg_const = 0.05

softmax_RICE = Softmax(n_class_RICE, lr, n_epochs, reg_const)
softmax_RICE.train(X_train_RICE, y_train_RICE)

In [710]:
pred_softmax = softmax_RICE.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_softmax, y_train_RICE)))

The training accuracy is given by: 99.844194


### Validate Softmax on Rice

In [711]:
pred_softmax = softmax_RICE.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_softmax, y_val_RICE)))

The validation accuracy is given by: 99.780038


### Testing Softmax on Rice

In [712]:
pred_softmax = softmax_RICE.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_softmax, y_test_RICE)))

The testing accuracy is given by: 99.752543


# Logistic Classifier

The Logistic Classifier has 2 hyperparameters that you can experiment with:
- **Learning rate** - similar to as defined above in Perceptron, this parameter scales by how much the weights are changed according to the calculated gradient update. 
- **Number of Epochs** - As described for perceptron.
- **Threshold** - The decision boundary of the classifier.


You will implement the Logistic Classifier in the **models/logistic.py**

The following code: 
- Creates an instance of the Logistic classifier class 
- The train function of the Logistic class is trained on the training data
- We use the predict function to find the training accuracy as well as the testing accuracy

### Training Logistic Classifer

In [727]:
for learning_rate in (0.5, 0.05):
    for n_epochs in (10, 12):
        for threshold in (0.5, 0.6, 0.7):
            print(learning_rate, n_epochs, threshold)
            lr = Logistic(learning_rate, n_epochs, threshold)
            lr.train(X_train_RICE, y_train_RICE)
            pred_lr = lr.predict(X_train_RICE)
            print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_RICE)))
            pred_lr = lr.predict(X_val_RICE)
            print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_RICE)))
            pred_lr = lr.predict(X_test_RICE)
            print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_RICE)))
            print("\n")

0.5 10 0.5
The training accuracy is given by: 99.770873
The validation accuracy is given by: 99.780038
The testing accuracy is given by: 99.587572


0.5 10 0.6
The training accuracy is given by: 99.770873
The validation accuracy is given by: 99.752543
The testing accuracy is given by: 99.587572


0.5 10 0.7
The training accuracy is given by: 99.770873
The validation accuracy is given by: 99.752543
The testing accuracy is given by: 99.587572


0.5 12 0.5
The training accuracy is given by: 99.871689
The validation accuracy is given by: 99.835029
The testing accuracy is given by: 99.780038


0.5 12 0.6
The training accuracy is given by: 99.670058
The validation accuracy is given by: 99.725048
The testing accuracy is given by: 99.560077


0.5 12 0.7
The training accuracy is given by: 99.871689
The validation accuracy is given by: 99.835029
The testing accuracy is given by: 99.780038


0.05 10 0.5
The training accuracy is given by: 99.752543
The validation accuracy is given by: 99.725048
Th

In [728]:
learning_rate = 0.5
n_epochs = 10
threshold = 0.9

lr = Logistic(learning_rate, n_epochs, threshold)
lr.train(X_train_RICE, y_train_RICE)

In [729]:
pred_lr = lr.predict(X_train_RICE)
print('The training accuracy is given by: %f' % (get_acc(pred_lr, y_train_RICE)))

The training accuracy is given by: 99.770873


### Validate Logistic Classifer

In [730]:
pred_lr = lr.predict(X_val_RICE)
print('The validation accuracy is given by: %f' % (get_acc(pred_lr, y_val_RICE)))

The validation accuracy is given by: 99.780038


### Test Logistic Classifier

In [731]:
pred_lr = lr.predict(X_test_RICE)
print('The testing accuracy is given by: %f' % (get_acc(pred_lr, y_test_RICE)))

The testing accuracy is given by: 99.587572
